In [151]:
import glob, os, json
from pathlib import Path
import itertools
from tqdm import tqdm
import spacy, nltk
from nltk import pos_tag, sent_tokenize
from itertools import chain
from collections import Counter
import time
import re
import itertools
from collections import Counter
import numpy as np
import pandas as pd

In [152]:
def remove_emojis(text):
    try:
        # UCS-4
        highpoints = re.compile(u'[\U00010000-\U0010ffff]')
    except re.error:
        # UCS-2
        highpoints = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')

    text = highpoints.sub('', text)
    try: 
        highpoints = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
    except:
        highpoints = re.compile(u'([\u2600-\u27BF])|([\uD83C][\uDF00-\uDFFF])|([\uD83D][\uDC00-\uDE4F])|([\uD83D][\uDE80-\uDEFF])')

    return highpoints.sub('', text)

In [145]:
files = sorted(glob.glob(os.path.join("gsDataAll/gsDataEn", "*.npy")))

In [4]:
NEED_VERBS = ["need", "needs", "needed", "needing", "necessitate", "necessitates", "necessitated", "necessitating",
               "require", "requires", "required", "requiring", "asking for", "ask for", "asks for", "asked for",
               "want", "wants", "wanted", "wanting", "demanding", "demand", "demands", "demanded", "request",
               "requests", "requesting", "requested"]
ASK_VERBS = ["ask", "asks", "asked", "asking"]
NEED_NOUNS = ["need", "needs", "want", "wants", "requirement", "requirements", "necessity", "necessities",
              "request", "requests", "demands", "demand"]

In [172]:
sents = {}
for file in tqdm(files):
    day = np.load(file, allow_pickle=True)
    tweets = pd.json_normalize(day)
    to_drop = [i for i, tweet in tweets.iterrows() if any(word in tweet["text"].lower() for word in NEED_VERBS) == False]
    tweets = tweets.drop(to_drop)
    tweets["text"] = tweets["text"].apply(lambda x: ' '.join(re.sub("([@#][A-Za-z0-9_]+)|(\w+:\/\/\S+)"," ", x).split()))
    tweets["text"] = tweets["text"].apply(lambda x: remove_emojis(x))
    tweets["text"] = tweets["text"].apply(lambda x: re.sub(r'\$\w*', '', x))
    tweets["text"] = tweets["text"].apply(lambda x: re.sub(r'#', '', x))
    tweets["text"] = tweets["text"].apply(lambda x: re.sub('&amp;', '', x))
    tweets["text"] = tweets["text"].apply(lambda x: re.sub(r'https?:\/\/.*\/\w*', ' ', x))
    sents_for_file = [sent_tokenize(tweet) for tweet in tweets["text"]]
    sents[file] = sents_for_file

100%|█████████████████████████████████████████| 863/863 [42:30<00:00,  2.96s/it]


In [7]:
options = ["ner", "tok2vec", "lemmatizer", "textcat", "senter",
                                      "sentencizer", "attribute_ruler", "trainable_lemmatizer",
                                      "textcat_multilabel", "entity_ruler", "entity_linker"]

In [ ]:
possibilities = []
for num in range(11):
    for subset in tqdm(list(itertools.combinations(options, num + 1))):
        start = time.time()
        nlp = spacy.load("en_core_web_sm", disable=subset)
        pos_tokenized = []
        for key, file in sents.items():
            for idx, tweet in enumerate(file):
                if idx == 5:
                    end = time.time() - start
                    break
                for sent in tweet:
                    string = string.replace("[^\w\d'\s-]+", ' ')
                    doc = nlp(re.sub(string)
                    pos_tokenized.append([{"Text": token.text, "POS": token.pos_,
                                   "Dep": token.dep_, "Tag": token.tag_, 
                                   "Children": [str(child) for child in token.children]} for token in doc])
        if pos_tokenized[0][19]["POS"] != '' and pos_tokenized[0][19]["Dep"] != '' and len(pos_tokenized[0][19]["Children"]) == 2:
            possibilities.append({"Options": subset, "ItPerSecond": 1 / (end / 5)})

In [180]:
fastest = min([subset["ItPerSecond"] for subset in possibilities])
options = [subset["Options"] for subset in possibilities if subset["ItPerSecond"] == fastest][0]

In [173]:
options = ('ner', 'lemmatizer', 'textcat', 'senter', 'sentencizer', 'trainable_lemmatizer', 'textcat_multilabel', 'entity_ruler', 'entity_linker')
nlp = spacy.load("en_core_web_sm", disable=options)
pos_tokenized = []
for key, file in tqdm(sents.items()):
    for idx, tweet in enumerate(file):
        for sent in tweet:
            string = sent.replace(r"[^\w\d'\s-]+", ' ')
            string = " ".join(string.split())
            # doc = nlp(string)
            # string = string.replace("/\s\s+/g", ' ')
            doc = nlp(re.sub(r"[^\w\d'\s-]+", '', string))
            pos_tokenized.append([{"Text": token.text, "POS": token.pos_,
                           "Dep": token.dep_, "Tag": token.tag_, 
                           "Children": [str(child) for child in token.children]} for token in doc])

100%|███████████████████████████████████████| 863/863 [3:15:14<00:00, 13.57s/it]


In [47]:
NEED_VERBS = ["need", "needs", "needed", "needing", "necessitate", "necessitates", "necessitated", "necessitating",
               "require", "requires", "required", "requiring", "want", "wants", "wanted", "wanting",
               "demanding", "demand", "demands", "demanded", "request", "requests", "requesting", "requested"]

In [8]:
def who_needs_what(sent: list):
    need_term = ''
    for idx, word in enumerate(sent):
        if (word["Text"] in NEED_VERBS or word["Text"] in ASK_VERBS) and word["POS"] == "VERB":
            need_term = word["Text"]
            what = verb_find_what(sent, idx)
            who = verb_find_who(sent, idx)
            break
        elif word["Text"] in NEED_NOUNS and word["POS"] == "NOUN":
            need_term = word["Text"]
            what = noun_find_what(sent, idx)
            who = noun_find_who(sent, idx)
            break
            
    return {"Who": who, "Need-Term": need_term, "What": what, "Need-POS": word["POS"]}

In [37]:
def get_adjectives(sent: list, cur_idx: int) -> str:
    adj = ''
    i = 1
    while(cur_idx-i >= 0):
        if sent[cur_idx-i]["Dep"] == "compound" or sent[cur_idx-i]["Dep"] == "amod":
            adj = (sent[cur_idx-i]["Text"] + ' ') + adj
        else:
            break
        i += 1
        
    return adj

In [10]:
def verb_find_what(sent: list, word_idx: int):
    adj = ''
    what = ''
    for cur_idx, word in enumerate(sent[word_idx:]):
        if (word["Dep"] == "dobj" or word["Dep"] == "pobj") and word["POS"] == "NOUN":
            what = word["Text"]
            adj = get_adjectives(sent, cur_idx)
        elif word["Dep"] == "dobj" and word["Text"] in sent[word_idx]["Children"] and word["POS"] == "PROPN":
            what = word["Text"]
                
    return adj + what

In [11]:
def verb_find_who(sent: list, word_idx: int):
    who = ''
    i = word_idx
    while (i != -1):
        if sent[i]["Dep"] == "nsubj" and sent[i]["Text"] in sent[word_idx]["Children"]:
            if sent[i]["Tag"] == "WDT":
                for word in sent[:i]:
                    if (word["POS"] == "NOUN" or word["POS"] == "PROPN") and word["Dep"] == "attr":
                        who = word["Text"]
            else:
                who = sent[i]["Text"]
        i -= 1
        
    return who

In [12]:
def noun_find_what(sent: list, word_idx: int):
    what = ''
    for cur_idx, word in enumerate(sent[word_idx:]):
        if word["Dep"] == "pobj" and word["POS"] == "NOUN":
            what = word["Text"]
            adj = get_adjectives(sent, cur_idx)
            
    return adj + what

In [13]:
def noun_find_who(sent: list, word_idx: int):
    who = ''
    i = word_idx
    dep = "nsubj"
    while (i != -1):
        if sent[i]["Tag"] == "WDT":
            dep = "attr"
        if sent[i]["Dep"] == dep:
            who = sent[i]["Text"]
            
        i -= 1
            
    return who

In [178]:
start = time.time()
triples, fails = [], []
for i, sent in enumerate(tqdm(pos_tokenized)):
    try:
        triples.append((who_needs_what(sent), i)) 
    except:
        fails.append(sent)
end = time.time() - start

In [ ]:
np.save("who_needs_what_output.npy", [triple[0] for triple in triples])

In [184]:
end

246.23144698143005

In [182]:
stats, totals = {}, {}
for verb in NEED_VERBS:
    stats[(verb, "VERB")] = 0
    totals[(verb, "VERB")] = 0
    
for verb in ASK_VERBS:
    stats[(verb, "VERB")] = 0
    totals[(verb, "VERB")] = 0

for noun in NEED_NOUNS:
    stats[(noun, "NOUN")] = 0
    totals[(noun, "NOUN")] = 0

In [183]:
for triple in tqdm(triples):
    pos = triple[0]["Need-POS"]
    totals[(triple[0]["Need-Term"], pos)] += 1
    if triple[0]["What"] != '' and triple[0]["Who"] != '':
        stats[(triple[0]["Need-Term"], pos)] += 1

100%|███████████████████████████████| 1506664/1506664 [04:56<00:00, 5080.74it/s]


In [186]:
proportions = [(value / totals[key]) if totals[key] != 0 else -1 for key, value in zip(stats.keys(), stats.values())]
proportions = {key: proportions[i] for i, key in enumerate(stats.keys())}

In [ ]:
json.dump(stats, "count_by_term.json")
json.dump(proportions, "prop_by_term.json")

In [21]:
triples[0]

({'Who': 'Ukraine',
  'Need-Term': 'needs',
  'What': 'guarantees',
  'Need-POS': 'VERB'},
 0)

In [16]:
who_list = [sent[0]["Who"] for sent in triples]
what_list = [sent[0]["What"] for sent in triples]

In [17]:
test = Counter(what_list)